In [60]:
import sys
import os
import numpy as np
from keras.models import sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
import pandas as pd

In [61]:
df = pd.read_csv("D:\\Projects\\Emotion Recognition\\emotion recognition dataset\\fer2013.csv")

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [63]:
df.describe()

,emotion
count,35887.000000
mean,3.323265
std,1.873819
min,0.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,6.000000


In [64]:
df.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [65]:
print(df["Usage"].value_counts())

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64


In [66]:
X_train, X_test, y_train, y_test = [],[],[],[]

for index,row in df.iterrows():
    val = row["pixels"].split(" ")
    try:
        if "Training" in row["Usage"]:
            X_train.append(np.array(val,"float32"))
            y_train.append(row["emotion"])
            
        elif "PublicTest" in row["Usage"]:
            X_test.append(np.array(val,"float32"))
            y_test.append(row["emotion"])
            
    except:
        print(f"error occured at index: {index} and row: {row}")
        
        
print(f"X_train sample data:{X_train[0:2]}")
print(f"X_test sample data: {X_test[0:2]}")
print(f"y_train sample data: {y_train[0:2]}")
print(f"y_test sample data: {y_test[0:2]}")

X_train sample data:[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
X_test sample data: [array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
y_train sample data: [0, 0]
y_test sample data: [0, 1]


In [67]:
X_train = np.array(X_train,"float32")
y_train = np.array(y_train,"float32")
X_test = np.array(X_test,"float32")
y_test = np.array(y_test,"float32")

In [68]:
#Normalizing data between 0 and 1

X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

In [69]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width,height = 48,48

X_train = X_train.reshape(X_train.shape[0],width, height,1)

X_test = X_test.reshape(X_test.shape[0],width, height,1)

In [71]:
X_train.shape

(28709, 48, 48, 1)

In [72]:
X_train[10].shape#after reshape

(48, 48, 1)

In [73]:
#Designing in CNN

model = sequential.Sequential()

#1st layer
model.add(Conv2D(num_features, kernel_size=(3,3),activation="relu",input_shape =(X_train.shape[1:])))
model.add(Conv2D(num_features, kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))


#2nd convolutional layer
model.add(Conv2D(num_features, kernel_size=(3,3),activation="relu"))
model.add(Conv2D(num_features, kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

#3rd convolutional layer
model.add(Conv2D(2*num_features, kernel_size=(3,3),activation="relu"))
model.add(Conv2D(2*num_features, kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

model.add(Dense(2*2*2*2*num_features,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(num_labels,activation="softmax"))

model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(),metrics=['accuracy'])




In [74]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,y_test),shuffle=True)


# fer_json = model.to_json()
# with open("fer.json","w") as json_file:
#     json_file.write(fer_json)

# model.save_weights("fer.h5")


fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

Epoch 1/30
449/449 [==============================] - 24s 50ms/step - loss: 1.7007 - accuracy: 0.3096 - val_loss: 1.5111 - val_accuracy: 0.4051
Epoch 2/30
449/449 [==============================] - 22s 48ms/step - loss: 1.4630 - accuracy: 0.4290 - val_loss: 1.3736 - val_accuracy: 0.4826
Epoch 3/30
449/449 [==============================] - 22s 49ms/step - loss: 1.3483 - accuracy: 0.4792 - val_loss: 1.3064 - val_accuracy: 0.4987
Epoch 4/30
449/449 [==============================] - 24s 52ms/step - loss: 1.2866 - accuracy: 0.5083 - val_loss: 1.2385 - val_accuracy: 0.5274
Epoch 5/30
449/449 [==============================] - 24s 52ms/step - loss: 1.2400 - accuracy: 0.5249 - val_loss: 1.2071 - val_accuracy: 0.5492
Epoch 6/30
449/449 [==============================] - 23s 52ms/step - loss: 1.1960 - accuracy: 0.5448 - val_loss: 1.2058 - val_accuracy: 0.5522
Epoch 7/30
449/449 [==============================] - 23s 51ms/step - loss: 1.1602 - accuracy: 0.5560 - val_loss: 1.1814 - val_accuracy: